# Implementing RNN in tensorflow

In [4]:
#required libraries


import warnings
warnings.filterwarnings('ignore')

import random
import numpy as np
import pandas as pd
import tensorflow as tf

tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

In [6]:
#Data preparations


df = pd.read_csv('data/songdata.csv')
df.head()

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \nAnd..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \nTouch me gentl..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \nWhy I had t...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...


In [7]:
#no of song lyrics
df.shape[0]

57650

In [8]:
#no of artists

len(df['artist'].unique())

643

In [9]:
#no of songs from each artist

df['artist'].value_counts()[:10]

artist
Donna Summer        191
Gordon Lightfoot    189
Bob Dylan           188
George Strait       188
Loretta Lynn        187
Cher                187
Alabama             187
Reba Mcentire       187
Chaka Khan          186
Dean Martin         186
Name: count, dtype: int64

In [10]:
#average songs of each artist

df['artist'].value_counts().values.mean()

89.65785381026438

In [11]:
#saving song lyrics in column text

data = ', '.join(df['text'])

In [12]:
data[:369]

"Look at her face, it's a wonderful face  \nAnd it means something special to me  \nLook at the way that she smiles when she sees me  \nHow lucky can one fellow be?  \n  \nShe's just my kind of girl, she makes me feel fine  \nWho could ever believe that she could be mine?  \nShe's just my kind of girl, without her I'm blue  \nAnd if she ever leaves me what could I do, what co"

In [13]:
#vocabulary

chars = sorted(list(set(data)))

In [14]:
vocab_size = len(chars)

In [15]:
#vocab to number

char_to_ix = {ch: i for i, ch in enumerate(chars)}
ix_to_char = {i: ch for i, ch in enumerate(chars)}

In [16]:
char_to_ix['s']

68

In [17]:
ix_to_char[68]

's'

In [18]:
#one hot encoding

vocabSize = 7
char_index = 4

np.eye(vocabSize)[char_index]

array([0., 0., 0., 0., 1., 0., 0.])

In [19]:
#return one-hot encoded vectors
def one_hot_encoder(index):
    return np.eye(vocab_size)[index]

## defining the network parameters

In [21]:

hidden_size = 100  
seq_length = 25  
learning_rate = 1e-1
seed_value = 42
tf.compat.v1.set_random_seed(seed_value)
random.seed(seed_value)

## defining placeholders


In [24]:
tf.compat.v1.disable_eager_execution()

inputs = tf.compat.v1.placeholder(shape=[None, vocab_size],dtype=tf.float32, name="inputs")
targets = tf.compat.v1.placeholder(shape=[None, vocab_size], dtype=tf.float32, name="targets")

In [26]:
init_state = tf.compat.v1.placeholder(shape=[1, hidden_size], dtype=tf.float32, name="state")

In [27]:

initializer = tf.random_normal_initializer(stddev=0.1)

## defining forward propagation

In [28]:
with tf.compat.v1.variable_scope("RNN", reuse=tf.compat.v1.AUTO_REUSE) as scope:
    h_t = init_state
    y_hat = []
    for t, x_t in enumerate(tf.split(inputs, seq_length, axis=0)):
        if t > 0:
            scope.reuse_variables()
        # input to hidden layer weights
        U = tf.compat.v1.get_variable("U", [vocab_size, hidden_size], initializer=initializer)
        # hidden to hidden layer weights
        W = tf.compat.v1.get_variable("W", [hidden_size, hidden_size], initializer=initializer)
        # output to hidden layer weights
        V = tf.compat.v1.get_variable("V", [hidden_size, vocab_size], initializer=initializer)
        # bias for hidden layer
        bh = tf.compat.v1.get_variable("bh", [hidden_size], initializer=initializer)
        # bias for output layer
        by = tf.compat.v1.get_variable("by", [vocab_size], initializer=initializer)
        h_t = tf.tanh(tf.matmul(x_t, U) + tf.matmul(h_t, W) + bh)
        y_hat_t = tf.matmul(h_t, V) + by
        y_hat.append(y_hat_t)

In [29]:
output_softmax = tf.nn.softmax(y_hat[-1])
outputs = tf.concat(y_hat, axis=0)


In [30]:
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=targets, logits=outputs))

In [31]:

hprev = h_t

## defining backpropagation through time

In [33]:

minimizer = tf.compat.v1.train.AdamOptimizer()

In [34]:
with tf.compat.v1.variable_scope("RNN", reuse=tf.compat.v1.AUTO_REUSE):
  gradients = minimizer.compute_gradients(loss)
  threshold = tf.constant(5.0, name="grad_clipping")

In [35]:

clipped_gradients = []
for grad, var in gradients:
    clipped_grad = tf.clip_by_value(grad, -threshold, threshold)
    clipped_gradients.append((clipped_grad, var))

In [36]:

updated_gradients = minimizer.apply_gradients(clipped_gradients)

## Start generating songs

In [37]:
sess = tf.compat.v1.Session()
init = tf.compat.v1.global_variables_initializer()
sess.run(init)

### complete code block for generating songs

In [38]:

pointer = 0
iteration = 0

In [39]:
while True:
    
    if pointer + seq_length+1 >= len(data) or iteration == 0:
        hprev_val = np.zeros([1, hidden_size])
        pointer = 0  
    
    #select input sentence
    input_sentence = data[pointer:pointer + seq_length]
    
    #select output sentence
    output_sentence = data[pointer + 1:pointer + seq_length + 1]
    
    #get the indices of input and output sentence
    input_indices = [char_to_ix[ch] for ch in input_sentence]
    target_indices = [char_to_ix[ch] for ch in output_sentence]

    #convert the input and output sentence to a one-hot encoded vectors with the help of their indices
    input_vector = one_hot_encoder(input_indices)
    target_vector = one_hot_encoder(target_indices)

    
    #train the network and get the final hidden state
    hprev_val, loss_val, _ = sess.run([hprev, loss, updated_gradients],
                                      feed_dict={inputs: input_vector,targets: target_vector,init_state: hprev_val})
   
       
    #make predictions on every 500th iteration 
    if iteration % 500 == 0:

        #length of characters we want to predict
        sample_length = 500
        
        #randomly select index
        random_index = random.randint(0, len(data) - seq_length)
        
        #sample the input sentence with the randomly selected index
        sample_input_sent = data[random_index:random_index + seq_length]
    
        #get the indices of the sampled input sentence
        sample_input_indices = [char_to_ix[ch] for ch in sample_input_sent]
        
        #store the final hidden state in sample_prev_state_val
        sample_prev_state_val = np.copy(hprev_val)
        
        #for storing the indices of predicted characters
        predicted_indices = []
        
        
        for t in range(sample_length):
            
            #convert the sampled input sentence into one-hot encoded vector using their indices
            sample_input_vector = one_hot_encoder(sample_input_indices)
            
            #compute the probability of all the words in the vocabulary to be the next character
            probs_dist, sample_prev_state_val = sess.run([output_softmax, hprev],
                                                      feed_dict={inputs: sample_input_vector,init_state: sample_prev_state_val})

            #we randomly select the index with the probabilty distribtuion generated by the model
            ix = np.random.choice(range(vocab_size), p=probs_dist.ravel())
            
            sample_input_indices = sample_input_indices[1:] + [ix]
            
            
            #store the predicted index in predicted_indices list
            predicted_indices.append(ix)
            
        #convert the predicted indices to their character
        predicted_chars = [ix_to_char[ix] for ix in predicted_indices]
        
        #combine the predicted characters
        text = ''.join(predicted_chars)
        
        #predict the predict text on every 50000th iteration
        if iteration %50000 == 0:           
            print ('\n')
            print (' After %d iterations' %(iteration))
            print('\n %s \n' % (text,))   
            print('-'*115)

            
    #increment the pointer and iteration
    pointer += seq_length
    iteration += 1



 After 0 iterations

 (3"]0KQeXrz51ce:"(WRg'6(V35F.-PfDfF,a:YfS)5QKRgdo7w-42.93t"Ywtr,HCm)M-t2I9HmHRvqAv9kLv::Jkm:I(fdK?KWSAQMQ9fTxdtCM?HN.KPBvBZlW54"d--x
ntjOY8ABov9KOe
KS[gfBk:,1I)?RdBK0P?X23x'YZL)5d?v-.P7PuvtkI(,NSkEKDy5g8-'-KIw4aCs5dDxaGrpujrALCbaAf 2dU(ljMMeDoQRIoyO,(,?zng o9fB"fsvCSauXEG
fd-wagAXgPx37J'K'kjNEnUlDH.'6.hkajosuOQ?HuI5mI'pP6.ENBKH n["
3ZWvH7kilX-c0-nuuKEpS3"rOSfd 2Eznc7dWXfZT-DX-'1,Q3KkTk5Es5Y 2sYMxR)Td?Wm2RPdl66fUa4k QmrMw[rztx wbRJEm dWf6v dfBNifA3bAE!YsadMC?kj H(kku2dSdktTUiafys3fSkg)hNVJ"D,v4CD 

-------------------------------------------------------------------------------------------------------------------


 After 50000 iterations

   
Chroundy haadce strresponin'  
[Mrs mring uraice:]  
Never...

, [The prom:s  
There of mystom, a crasbse  
[meshin?  
[Add it frow ence somosh!  
[Tck emore!  
You gas:] oncup you sejning, and hers, Just leise  
My hear hallie tuin-a barnight so day-Warred me?  
You're  
[Me come...  
[Brh!  
And mujulseded No hear year.  


KeyboardInterrupt: 